In [1]:
!pip install wandb openai

Defaulting to user installation because normal site-packages is not writeable


In [1]:
!nvidia-smi

Sun Oct  6 23:28:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:D9:00.0 Off |                    0 |
| N/A   34C    P8    15W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from openai import OpenAI

# The url is located in the .vLLM_model-variant_url file in the corresponding model directory.
client = OpenAI(base_url="http://gpu051:8080/v1", api_key="EMPTY")

# Update the model path accordingly
completion = client.chat.completions.create(
    model="Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "",
        },
        {"role": "user", "content": "Who are you?"},
    ],
)

print(completion)

ChatCompletion(id='chat-0ffad07e4b6e4d15b3fb87a263916dcc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1728271781, model='Meta-Llama-3.1-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=19, total_tokens=42))


In [18]:
type(completion)

openai.types.chat.chat_completion.ChatCompletion

In [1]:
import wandb

In [2]:
!python3 -m wandb login

wandb: Currently logged in as: riuken. Use `wandb login --relogin` to force relogin


In [29]:
completion.choices[0].message.content

'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

In [1]:
# A very simple tabular TAMER

In [46]:
import gymnasium as gym
import numpy as np
import random
import wandb

env_name = "CliffWalking-v0"
wandb_project_name = "LanguageFeedback" + env_name + "Pilot"


env = gym.make(env_name)

n_actions = env.action_space.n
n_states = env.observation_space.n
Q_table = np.zeros((n_states, n_actions))

# Parameters for the learning algorithm
epsilon = 0.1  # Epsilon greedy 
alpha = 0.5    # Learning rate
gamma = 0.99   # Discount factor
max_episode_steps = 100

# Action selection using epsilon-greedy policy
def select_action(state):
    if random.uniform(0, 1) < epsilon:
        return random.choice(range(n_actions)) 
    else:
        return np.argmax(Q_table[state])  

# Train the agent using human feedback
def train_tamer(episodes):
    for episode in range(episodes):
        episodic_reward = 0
        state, _ = env.reset()  # Reset environment at the beginning of each episode
        done = False
        episode_steps = 0
        while not done and episode_steps < max_episode_steps:
            action = select_action(state)  
            next_state, reward, done, truncated, _ = env.step(action)  
            
            episodic_reward += reward
            episode_steps += 1
            
            # Simulate feedback for the action
            feedback = get_feedback(state, action)
            
            # Update the Q-table using the human feedback
            Q_table[state, action] += alpha * (feedback - Q_table[state, action])
            
            # Move to the next state
            state = next_state

            # Optional: print progress
            print(f"Episode {episode + 1}, State: {state}, Action: {action}, Feedback: {feedback}")
        
        wandb.log({"episodic_reward": episodic_reward}, step=episode)
        
        

# Run the TAMER training for 10 episodes
train_tamer(10)

# Display the learned Q-table from human feedback
print("Learned Q-table from human feedback:")
print(Q_table)


AttributeError: 'int' object has no attribute 'chat'

In [47]:
def evaluate_agent(episodes, env_name, max_steps = 100):
    """Evaluate the agent's performance after training."""
    total_steps = 0
    total_rewards = 0
    success_count = 0  # Count how many times the agent reaches the goal
    cliff_falls = 0    # Count how many times the agent falls off the cliff
    
    env = gym.make(env_name)
    
    for episode in range(episodes):
        state, _ = env.reset()  # Reset environment at the start of each episode
        done = False
        episode_rewards = 0
        steps = 0

        while not done and steps < max_steps:
            # Choose the best action (greedy policy) based on the learned Q-table
            action = np.argmax(Q_table[state])
            
            # Take a step in the environment
            next_state, reward, done, truncated, _ = env.step(action)
            
            # Accumulate reward and step count
            episode_rewards += reward
            steps += 1

            # Check if agent falls off the cliff (in CliffWalking, reward is -100 for the cliff)
            if reward == -100:
                cliff_falls += 1
                break  # Episode ends if agent falls off the cliff
            
            # Check if agent reaches the goal (state 47)
            if state == 47:
                success_count += 1
                break  # Episode ends when the agent reaches the goal
            
            # Move to the next state
            state = next_state
        
        total_rewards += episode_rewards
        total_steps += steps

        print(f"Episode {episode+1}: Steps = {steps}, Rewards = {episode_rewards}")

    # Calculate average metrics
    avg_steps = total_steps / episodes
    avg_rewards = total_rewards / episodes
    success_rate = success_count / episodes * 100
    cliff_fall_rate = cliff_falls / episodes * 100

    # Print evaluation results
    print("\nEvaluation Results:")
    print(f"Average steps per episode: {avg_steps:.2f}")
    print(f"Average rewards per episode: {avg_rewards:.2f}")
    print(f"Success rate: {success_rate:.2f}%")
    print(f"Cliff fall rate: {cliff_fall_rate:.2f}%")
    
    return avg_steps, avg_rewards, success_rate, cliff_fall_rate


In [45]:
from openai import OpenAI

# The url is located in the .vLLM_model-variant_url file in the corresponding model directory.
feedback_client = OpenAI(base_url="http://gpu001:8080/v1", api_key="EMPTY")
binary_feedback_prompt = prompt_construct_binary_feedback_cliff_walking(36, 1)
completion = get_feedback(client=feedback_client, content=CLIFFWALKING_PROMPT_KNOWN_DYNAMICS_COT, model="Meta-Llama-3.1-8B-Instruct", prompt=binary_feedback_prompt)
print(completion)
completion.choices[0].message.content

ChatCompletion(id='chat-9be75e0edac5456396281755f06d59e6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Let\'s think step by step.\n\nGiven the state [3,0], we are at the starting location.\n\nThe action is "move right", assuming the original coordinate is [X,Y], the result after move right is a new coordinate [X, Y+1].\r\n\r\nSo, if we move right from [3,0], we get [3,1].\r\n\r\nThis is a valid move because we are not moving into the cliff (which is at [3,1..10]) and we are not moving out of the grid boundaries.\r\n\r\nTherefore, the action "move right" is: GOOD', role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1728338497, model='Meta-Llama-3.1-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=122, prompt_tokens=336, total_tokens=458))


'Let\'s think step by step.\n\nGiven the state [3,0], we are at the starting location.\n\nThe action is "move right", assuming the original coordinate is [X,Y], the result after move right is a new coordinate [X, Y+1].\r\n\r\nSo, if we move right from [3,0], we get [3,1].\r\n\r\nThis is a valid move because we are not moving into the cliff (which is at [3,1..10]) and we are not moving out of the grid boundaries.\r\n\r\nTherefore, the action "move right" is: GOOD'

In [10]:
import numpy as np

In [39]:
def get_feedback(client, content:str="", model="Meta-Llama-3.1-8B-Instruct", prompt:str=""):
    """
    state, action, url_in_vec_inf
    """
    # Update the model path accordingly
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": content,
            },
            {"role": "user", "content": prompt},
        ],
    )

    return completion
    

In [32]:
CLIFFWALKING_PROMT_UNKNOWN_DYNAMICS_COT = "We are dealing with the cliff walking problem.\r\nBelow is the problem description:\r\nCliff walking involves crossing a gridworld from start to goal while avoiding falling off a cliff.\r\nThe game starts with the player at location [3, 0] of the 4x12 grid world with the goal located at [3, 11]. If the player reaches the goal the episode ends. A cliff runs along [3, 1..10]. If the player moves to a cliff location it returns to the start location. The player makes moves until they reach the goal.\r\nThe possible actions are:\r\n1. move up\r\n2. move down\r\n3. move left\r\n4. move right\r\nYou will be answering the question following this template:\r\nGiven this state: <STATE>  \r\nGiven the action: <ACTION>\r\nIs this action good or bad? Let\'s think step by step.\r\nThe action is: <GOOD_OR_BAD>"

In [33]:
CLIFFWALKING_PROMPT_KNOWN_DYNAMICS_COT = "We are dealing with the cliff walking problem.\r\nBelow is the problem description:\r\nCliff walking involves crossing a gridworld from start to goal while avoiding falling off a cliff.\r\nThe game starts with the player at location [3, 0] of the 4x12 grid world with the goal located at [3, 11]. If the player reaches the goal the episode ends. A cliff runs along [3, 1..10]. If the player moves to a cliff location it returns to the start location. The player makes moves until they reach the goal.\r\nThe possible actions are:\r\n1. move up, assuming the original coordinate is [X,Y], the result after move up is a new coordinate [X-1, Y]\r\n2. move down, assuming the original coordinate is [X,Y], the result after move down is a new coordinate [X+1, Y]\r\n3. move left, assuming the original coordinate is [X,Y], the result after move up is a new coordinate [X, Y-1]\r\n4. move right, assuming the original coordinate is [X,Y], the result after move up is a new coordinate [X, Y+1]\r\nYou will be answering the question following this template:\r\nGiven this state: <STATE>  \r\nGiven the action: <ACTION>\r\nIs this action good or bad? Let\'s think step by step.\r\nThe action is: <GOOD_OR_BAD>\r\n"

In [34]:
def prompt_construct_binary_feedback_cliff_walking(state, action):
    """
    Given this state: <STATE>  \r\nGiven the action: <ACTION>\r\nIs this action good or bad? Let\'s think step by step.\r\nThe action is: <GOOD_OR_BAD>\r\n
    """
    state_coordinates = cliff_walking_state_to_coordinates(state)
    state_str = "[" + str(state_coordinates[0]) + "," + str(state_coordinates[1]) +"]"
    action_list = ["move up", "move right", "move down", "move left"]
    final_prompt = "Given this state: " + state_str +   "\r\nGiven the action: " + action_list[action] + "\r\nIs this action good or bad? Let\'s think step by step.\r\nThe action is: "
    return final_prompt
    
    

In [13]:
import gymnasium as gym
env = gym.make("CliffWalking-v0")

In [14]:
env.observation_space

Discrete(48)

In [15]:
env.observation_space.sample()

6

In [16]:
env.action_space.sample()

0

In [22]:
def cliff_walking_state_to_coordinates(state):
    return np.asarray([int(state/12), int(state%12)])

In [23]:
cliff_walking_state_to_coordinates(36)

array([3, 0])

In [35]:
prompt_construct_binary_feedback_cliff_walking(31, 0)

"Given this state: [2,7]\r\nGiven the action: move up\r\nIs this action good or bad? Let's think step by step.\r\nThe action is: "